In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

print("\n... IMPORTS STARTING ...\n")

print("\n\tVERSION INFORMATION")
# Machine Learning and Data Science Imports
import tensorflow as tf; print(f"\t\t– TENSORFLOW VERSION: {tf.__version__}");
import tensorflow_hub as tfhub; print(f"\t\t– TENSORFLOW HUB VERSION: {tfhub.__version__}");
import tensorflow_addons as tfa; print(f"\t\t– TENSORFLOW ADDONS VERSION: {tfa.__version__}");
import tensorflow_io as tfio; print(f"\t\t– TENSORFLOW I/O VERSION: {tfio.__version__}");
import pandas as pd; pd.options.mode.chained_assignment = None;
import numpy as np; print(f"\t\t– NUMPY VERSION: {np.__version__}");
import sklearn; print(f"\t\t– SKLEARN VERSION: {sklearn.__version__}");
from sklearn.preprocessing import RobustScaler, PolynomialFeatures
from pandarallel import pandarallel; pandarallel.initialize();
from sklearn.model_selection import GroupKFold, StratifiedKFold
from scipy.spatial import cKDTree

# # RAPIDS
# import cudf, cupy, cuml
# from cuml.neighbors import NearestNeighbors
# from cuml.manifold import TSNE, UMAP

# Built In Imports
from kaggle_datasets import KaggleDatasets
from collections import Counter
from datetime import datetime
from glob import glob
import openslide
import warnings
import requests
import hashlib
import imageio
import IPython
import sklearn
import urllib
import zipfile
import pickle
import random
import shutil
import string
import json
import math
import time
import gzip
import ast
import sys
import io
import os
import gc
import re

# Visualization Imports
from matplotlib.colors import ListedColormap
from matplotlib.patches import Rectangle
import matplotlib.patches as patches
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm; tqdm.pandas();
import plotly.express as px
import tifffile as tif
import seaborn as sns
from PIL import Image, ImageEnhance; Image.MAX_IMAGE_PIXELS = 5_000_000_000;
import matplotlib; print(f"\t\t– MATPLOTLIB VERSION: {matplotlib.__version__}");
from matplotlib import animation, rc; rc('animation', html='jshtml')
import plotly
import PIL
import cv2 as cv

import plotly.io as pio
print(pio.renderers)



def seed_it_all(seed=7):
    """ Attempt to be Reproducible """
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)


    
print("\n\n... IMPORTS COMPLETE ...\n")

In [ ]:
# Different interpolcasefoldation based on version of Kaggle Kernel
try:
    DEFAULT_INTERPOLATION = Image.resampling.BILINEAR
except Exception as AttributeError:
    DEFAULT_INTERPOLATION = Image.BILINEAR

In [ ]:
# Local path to training and validation images
DATA_DIR = "/kaggle/input/mayo-clinic-strip-ai"
save_locally = None
load_locally = None

print(f"\n... DATA DIRECTORY PATH IS:\n\t--> {DATA_DIR}")

print(f"\n... IMMEDIATE CONTENTS OF DATA DIRECTORY IS:")
for file in tf.io.gfile.glob(os.path.join(DATA_DIR, "*")): print(f"\t--> {file}")

print("\n\n... DATA ACCESS SETUP COMPLETED ...\n")

In [ ]:
print("\n... BASIC DATA SETUP STARTING ...\n\n")

# Open the training dataframe and display the initial dataframe
TRAIN_DIR = os.path.join(DATA_DIR, "train")
TRAIN_CSV = os.path.join(DATA_DIR, "train.csv")
train_df = pd.read_csv(TRAIN_CSV)
train_df["image_path"] = train_df.image_id.apply(lambda x: os.path.join(TRAIN_DIR, x+".tif"))

# Add size information to train dataframe
train_df["image_size"] = train_df.image_path.progress_apply(lambda x: Image.open(x).size)
train_df["image_width"] = train_df.image_size.apply(lambda x: int(x[0]))
train_df["image_height"] = train_df.image_size.apply(lambda x: int(x[1]))
train_df["image_area"] = train_df.image_width*train_df.image_height

print("\n... TRAINING DATAFRAME... \n")
display(train_df)

# Open the testing dataframe and display the initial dataframe
TEST_DIR = os.path.join(DATA_DIR, "test")
TEST_CSV = os.path.join(DATA_DIR, "test.csv")
test_df = pd.read_csv(TEST_CSV)
test_df["image_path"] = test_df.image_id.apply(lambda x: os.path.join(TEST_DIR, x+".tif"))

# Add size information to test dataframe
test_df["image_size"] = test_df.image_path.progress_apply(lambda x: Image.open(x).size)
test_df["image_width"] = test_df.image_size.apply(lambda x: int(x[0]))
test_df["image_height"] = test_df.image_size.apply(lambda x: int(x[1]))
test_df["image_area"] = test_df.image_width*test_df["image_height"]

print("\n... TESTING DATAFRAME... \n")
display(test_df)

# Open the other dataframe and display the initial dataframe
OTHER_DIR = os.path.join(DATA_DIR, "other")
OTHER_CSV = os.path.join(DATA_DIR, "other.csv")
other_df = pd.read_csv(OTHER_CSV)

print("\n... OTHER DATAFRAME... \n")
display(other_df)

# Open the sample submission dataframe
SS_CSV   = os.path.join(DATA_DIR, "sample_submission.csv")
ss_df = pd.read_csv(SS_CSV)

print("\n... SAMPLE SUBMISSION DATAFRAME... \n")
display(ss_df)

# For debugging purposes when the test set hasn't been substituted we will know
DEBUG=len(ss_df)==4
print(f"\n\n\n... ARE WE DEBUGGING: {DEBUG}... \n")

# Capture examples that span smallest, median, and large image size examples
SMALLEST_IMAGE_ID = train_df[train_df.image_area==train_df.image_area.min()].image_id.values[0]
SMALLEST_IMAGE_ROW = train_df[train_df.image_id==SMALLEST_IMAGE_ID]
MEDIAN_IMAGE_ID = train_df.sort_values(by="image_area").reset_index().iloc[len(train_df)//2].image_id
MEDIAN_IMAGE_ROW = train_df[train_df.image_id==MEDIAN_IMAGE_ID]
LARGEST_IMAGE_ID = train_df[train_df.image_area==train_df.image_area.max()].image_id.values[0]
LARGEST_IMAGE_ROW = train_df[train_df.image_id==LARGEST_IMAGE_ID]

print("\n... BASIC DATA SETUP FINISHED ...\n\n")

# 3  HELPER FUNCTION & CLASSES

In [ ]:
# Different interpolcasefoldation based on version of Kaggle Kernel
try:
    DEFAULT_INTERPOLATION = Image.resampling.BILINEAR
except Exception as AttributeError:
    DEFAULT_INTERPOLATION = Image.BILINEAR

In [ ]:
def flatten_l_o_l(nested_list):
    """ Flatten a list of lists """
    return [item for sublist in nested_list for item in sublist]

In [ ]:
def pil_open_downsampled_slide(img_path, downsample_by=8, interpolation=None, reducing_gap=3.0, as_numpy=True):
    """
    
    Helper function to convert WSI into smaller downscaled version using PIL
    
    Timing details for MAYO CLINIC STRIP AI dataset:
        SMALLEST IMAGE BY AREA (4417, 5314)
            * Function takes ~0.5 seconds to run
        MEDIAN IMAGE BY AREA   (17573, 38743)
            * Function takes ~16 seconds to run
        LARGEST IMAGE BY AREA  (48282, 101406)(~208X LARGER THAN SMALLEST IMAGE)(~7X LARGER THAN MEDIAN IMAGE)
            * Function will CRASH!
    
    Args:
        img_path (str): Path to .tif file to be downsampled
        downsample_by (int): How many times smaller should resultant 
            image be. i.e. image_size*(1/downsample_by) = new_size
        interpolation (int, optional): The value that enumerates
            the type of interpolation to be used in downsampling.
            This can be one
               * `Image.NEAREST`  --> ENUM=0
               * `Image.BOX`      --> ENUM=1
               * `Image.BILINEAR` --> ENUM=2
               * `Image.HAMMING`  --> ENUM=3
               * `Image.BICUBIC`  --> ENUM=4
               * `Image.LANCZOS`  --> ENUM=5
           If omitted, it defaults to `Image.BILINEAR`
       reducing_gap (float, optional): How many steps to use when interpolating/resampling
           the image after integer downscaling is performed. 
           ELI5: 
               * Higher (Approaching 5) means it takes longer but is better quality
               * Lower  (Approaching 0) means it is much faster but  lower  quality
        as_numpy (bool, optional): Whether to return image as numpy array (default)
           or leave as PIL.Image object for further manipulation
    
    Returns:
        Downsampled image as a numpy array of type uint8 with only 3 channels
    """
    
    # Catches and warning
    if downsample_by<8: print("\n... WARNING – DUE TO LOW DOWNSCALE_BY VALUE THE RETURND IMAGE MAY OCCUPY A LARGE AMOUNT OF RAM – WARNING ...\n")
    if interpolation is None: interpolation=DEFAULT_INTERPOLATION
        
    # Open the image with PIL
    tmp_img = Image.open(img_path)
    
    # Get original dimensions and calculate new dimensions
    original_dimensions = tmp_img.size
    downsample_dimensions = (int(original_dimensions[0]/downsample_by), int(original_dimensions[1]/downsample_by))
    
    # Resize the image
    tmp_img.thumbnail(size=downsample_dimensions, 
                      resample=interpolation,
                      reducing_gap=reducing_gap)
    
    # Return either np.ndarray or PIL.Image object
    return np.asarray(tmp_img) if as_numpy else tmp_img

In [ ]:
def get_slide_tiles(img_path, tile_size=(384,384), stride=(256, 256), downsample_by=4, drop_empty=True, drop_near_empty=True, near_empty_sum_thresh=2500, near_empty_bg_thresh=170, return_downsampled_image=False):
    """ Tile a WSI
    
    Args:
        img_path (str): Path to .tif file to be downsampled 
        tile_size (tuple of ints, optional): The dimensions of the tile/patches to be extracted
        stride (tuple of ints, optional): The stride between tile/patches. 
            i.e. a stride of two: xx____, __xx__, ____xx
        downsample_by (int, optional): How many times smaller should resultant 
            image be. i.e. image_size*(1/downsample_by) = new_size
        drop_empty (bool, optional): Whether or not to drop images that are determined to be empty
            i.e. ALL the pixels in the tile are determined to be background pixels
        drop_near_empty (bool, optional): Whether or not to drop images that are determined to be "mostly" empty
            i.e. MOST of the pixels in the tile are determined to be background pixels
        near_empty_sum_thresh (int, optional): The threshold to use to determine what "MOST"
            means in the context of background vs forgeround.
            i.e. How many pixels are allowed to be background pixels before we drop?
        near_empty_bg_thresh (int, optional): What qualifies as the threshold at which if the pixel
            is brighter than this it is considered background... wheras lower than this is foreground
            (I AM AWARE FOREGROUND WILL YIELD SOME PERCENTAGE OF "BACKGROUND" PIXELS)
        return_downsampled_image (bool, optional): Whether we return the downsampled numpy array
    
    Returns:
        A list of tiles – np.ndarray of shape `tile_size`
    """
    
    o_img = pil_open_downsampled_slide(img_path, downsample_by)
    (orig_h, orig_w), (tile_h, tile_w), (stride_w, stride_h) = o_img.shape[:-1], tile_size, stride
    
    tile_map, non_bg_sum = {}, near_empty_sum_thresh
    
    row_steps = range(0, orig_h+stride_h, stride_h)
    col_steps = range(0, orig_w+stride_w, stride_w)
    n_steps = len(row_steps)*len(col_steps)
    for j, y in enumerate(row_steps):
        for i, x in enumerate(col_steps):
            tile = o_img[y:(y+tile_h), x:(x+tile_w)]
            if drop_empty or drop_near_empty: 
                non_bg_sum = np.where(tile>near_empty_bg_thresh, 0, 1).sum()
            if (drop_empty and non_bg_sum==0) or (drop_near_empty and non_bg_sum<near_empty_sum_thresh): 
                if n_steps<=999: print(" XXX ", end="")
            else:
                loc_str = f" {j*len(col_steps)+i+1:>03} "
                tile_map[loc_str]=tile
                if n_steps<=999: print(loc_str, end="")
        if n_steps<=999: print("\n")
        
    if return_downsampled_image:
        return tile_map, o_img
    else:
        return tile_map


In [ ]:
def process_and_plot(pd_row, draw_grid=True, grid_stride=(256,256), grid_size=(384,384), draw_text=True):
    def _add_text(img, annot, color, pt1, pt2, **kwargs):
            text_width, text_height = cv.getTextSize(annot, FONT, FONT_SCALE, FONT_THICKNESS)[0]
            annot_loc = (int(round(pt1[0]+((pt2[0]-pt1[0])/2)-(text_width/2))), int(round(pt1[1]+((pt2[1]-pt1[1])/2)+(text_height/2))))
            img = cv.putText(img, annot, annot_loc, FONT, FONT_SCALE, color, FONT_THICKNESS, FONT_LINE_TYPE)
            return img
        
    print(f"\n... PROCESSING IMAGE {pd_row.image_id.values[0]} - {pd_row.image_size.values[0]} ...\n")
    t1 = time.time()
    image_tile_map, downsampled_image = get_slide_tiles(pd_row.image_path.values[0], return_downsampled_image=True)
    
    n_tiles = len(image_tile_map)
    print(f"\nIMAGE PROCESSING ELAPSED TIME: {time.time()-t1:.4f} SECONDS")
    print("RESULTANT NUMBER OF TILES: ", n_tiles)
    
    print("\n\n... ORIGINAL IMAGE - DOWNSAMPLED BY A FACTOR OF 4 ...\n\n")
    w_to_h_ratio = pd_row.image_width.values[0]/pd_row.image_height.values[0]
    plt.figure(figsize=(int(20*w_to_h_ratio), 20) if w_to_h_ratio>1 else (20, int(20/w_to_h_ratio)))
    plt.axis(False)
    plt.title(f"Complete Downsampled Image – {pd_row.image_id.values[0]}", fontweight="bold")
    if draw_grid:
        clr_cnt, c_map, clrs, draw_map_fg, draw_map_bg = 0, plt.get_cmap("rainbow"), [], [], []
        row_steps = range(0, downsampled_image.shape[0]+grid_stride[1], grid_stride[1])
        col_steps = range(0, downsampled_image.shape[1]+grid_stride[0], grid_stride[0])
        clrs = [[int(x*255) for x in c_map((i+1)/len(image_tile_map.keys()))] for i in range(len(image_tile_map.keys()))]
        draw_image = downsampled_image.copy()
        if draw_text: 
            draw_annots, FONT, FONT_SCALE, FONT_THICKNESS, FONT_LINE_TYPE = [], cv.FONT_HERSHEY_SIMPLEX, 1.56789, 3, cv.LINE_AA
            
        for j, y in enumerate(row_steps):
            for i, x in enumerate(col_steps):
                if f" {j*len(col_steps)+i+1:>03} " in image_tile_map.keys():
                    draw_map_fg.append(dict(pt1=(x,y), pt2=(x+grid_size[0], y+grid_size[1]), color=clrs[clr_cnt], thickness=3, annot=f" {j*len(col_steps)+i+1:>03} "))
                    clr_cnt +=1
                else:
                    draw_map_bg.append(dict(pt1=(x,y), pt2=(x+grid_size[0], y+grid_size[1]), color=(10,10,10), thickness=2, annot=f" {j*len(col_steps)+i+1:>03} "))                
                    
        for draw_map in draw_map_bg+draw_map_fg:
            draw_image = cv.rectangle(draw_image, draw_map["pt1"], draw_map["pt2"], draw_map["color"], draw_map["thickness"])
            if draw_text:
                _add_text(draw_image, draw_map["annot"], draw_map["color"], draw_map["pt1"], draw_map["pt2"])
    plt.imshow(downsampled_image if not draw_grid else draw_image)
    plt.tight_layout()
    plt.show()

    print(f"\n\n\n... PLOTTING FIRST {min(16, n_tiles)} OF {n_tiles} TILES FROM IMAGE ...\n")
    plt.figure(figsize=(20,20))
    for i, (loc_str, tile) in enumerate(image_tile_map.items()):
        if i==16: break
        plt.subplot(4,4,i+1)
        plt.title(f"Tile Location: {loc_str}", fontweight="bold")
        plt.axis(False)
        plt.imshow(tile)
    plt.tight_layout()
    plt.show()

# EDA

In [ ]:
# IMAGE_IDS
print(f"\n... THERE ARE {train_df.image_id.nunique()} UNIQUE VALUES FOR {len(train_df)} EXAMPLES IN THE TRAINING DATA ...")
print(f"... THERE ARE {test_df.image_id.nunique()} UNIQUE VALUES FOR {len(test_df)} EXAMPLES IN THE TEST DATA ...\n")

if train_df.image_id.nunique() == len(train_df): print(f"... NO DUPLICATE IDS IN TRAINING DATA ...") 
if test_df.image_id.nunique() == len(test_df): print(f"... NO DUPLICATE IDS IN TRAINING DATA ...") 

In [ ]:
# CENTER_ID
print(f"\n... `center_id` HAS {train_df['center_id'].nunique()} UNIQUE VALUES FOR {len(train_df)} EXAMPLES IN THE TRAINING DATA ...\n\n")
train_df["center_id_count"] = train_df.groupby("center_id")["image_id"].transform('count')
fig = px.histogram(train_df, "center_id", 
             color="center_id_count", 
             color_discrete_sequence=px.colors.sequential.Inferno_r,
             title="<b>Histogram Showing Frequency of `center_id` Values In Training Dataset</b>",
             labels={"color":"<b>Center ID Legend</b>"},
             )
fig.show()

In [ ]:
# PATIENT_ID
print(f"\n... `patient_id` HAS {train_df['patient_id'].nunique()} UNIQUE VALUES FOR {len(train_df)} EXAMPLES IN THE TRAINING DATA ...\n\n")
train_df["patient_id_count"] = train_df.groupby("patient_id")["image_id"].transform('count')
fig = px.histogram(train_df, "patient_id_count", color="center_id", 
             title="<b>Histogram Showing Frequency of `patient_id` Values In Training Dataset Coloured By Center ID</b>",
             labels={"center_id":"<b>Center ID Legend</b>"})
fig.show()

In [ ]:
# LABEL
fig = px.pie(train_df, "label", title="<b>Pie Chart Showing Class Imbalance In Provided Label Clot Etiology</b>", hole=0.30901699, )
fig.show()

fig = px.histogram(train_df, "image_area", color="label",
                title="<b>Scatter Plot Showing Class Distribution According to Slide Size</b>")
fig.show()

In [ ]:
# IMAGE_SIZE
display(train_df[(train_df.image_area==train_df.image_area.min()) | (train_df.image_area==train_df.image_area.max())])

fig = px.scatter(train_df, "image_width", "image_height", color="label", size="image_area",
                 title="<b>Scatter Plot Showing Class Distribution According to Slide Size</b>",
                 width=1200, height=1200)
fig.show()

# SAMPLES

In [ ]:
process_and_plot(SMALLEST_IMAGE_ROW)

In [ ]:
# process_and_plot(MEDIAN_IMAGE_ROW)

In [ ]:
# process_and_plot(LARGEST_IMAGE_ROW)